
Em Agosto de 2023 a NIST publicou um draft da norma FIPS203  para um Key Encapsulation Mechanism (KEM) derivado dos algoritmos KYBER. 
O preâmbulo do “draft”:

*A key-encapsulation mechanism (or KEM) is a set of algorithms that, under certain conditions, can be used by two parties to establish a shared secret key over a public channel. A shared secret key that is securely established using a KEM can then be used with symmetric-key cryptographic algorithms to perform basic tasks in secure communications, such as encryption and authentication. This standard specifes a key-encapsulation mechanism called ML-KEM. The security of ML-KEM is related to the computational diffculty of the so-called Module Learning with Errorsproblem. At present, ML-KEM is believed to be secure even against adversaries who possess a quantum computer*


Neste trabalho pretende-se implementar em Sagemath um protótipo deste standard parametrizado de acordo com as variantes sugeridas na norma (512, 768 e 1024 bits de segurança).

ML-KEM é um mecanismo de encapsulamento de chaves recentemente padronizado baseado em retículos.

Este utiliza o Módulo de Aprendizado com Erros como primitiva subjacente, que é uma variante estruturada de retículos que oferece bom desempenho e tamanhos de chave e texto cifrado relativamente pequenos e equilibrados. ML-KEM foi padronizado com três parâmetros: ML-KEM-512, ML-KEM-768 e ML-KEM-1024. Estes foram mapeados pelo NIST para os três níveis de segurança definidos no Projeto PQC do NIST: nível 1, 3 e 5. Esses níveis correspondem à dificuldade de quebrar AES-128, AES-192 e AES-256, respetivamente.

Esta especificação introduz o ML-KEM-768 e ML-KEM-1024 nas trocas de chaves do IKEv2 como parâmetros de nível de segurança conservadores que não terão impacto significativo no desempenho dos túneis IKEv2/IPsec, que geralmente permanecem ativos por longos períodos de tempo. Como os tamanhos de chave pública e texto cifrado do ML-KEM-768 e ML-KEM-1024 podem exceder o MTU típico da rede, essas trocas de chaves geralmente exigem dois ou três pacotes IP de rede tanto de quem inicia quanto do que responde.
(https://www.ietf.org/archive/id/draft-kampanakis-ml-kem-ikev2-01.html)
(https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.203.ipd.pdf)

In [690]:
from typing import List, Tuple
import hashlib
import os
from functools import reduce
from shakestream import ShakeStream
#import sage.all

O FIPS-203 especifica o ML-KEM, que foi baseado no único vencedor do KEM da Competição Pós-Quântica do NIST: Kyber. O ML-KEM define 3 conjuntos de parâmetros, cada um em um nível diferente de segurança:

1. ML-KEM-512 (equivalência de segurança ao AES-128, nível 1 de quebra)
2. ML-KEM-768 (equivalência de segurança ao AES-192, nível 3 de quebra)
3. ML-KEM-1024 (equivalência de segurança ao AES-256, nível 5 de quebra)

O ML-KEM é apropriado como uma substituição geral para algoritmos de troca de chaves vulneráveis ​​a ataques quânticos, como ECDH ou FFDH. É notável que ECDH e FFDH são algoritmos de Troca de Chave Não-Interativa (NIKE), mas ML-KEM não é, então, para aplicações onde a não-interatividade é um requisito, ML-KEM NÃO é uma substituição apropriada. Embora o desempenho do ML-KEM seja muito bom, os tamanhos dos artefatos criptográficos são maiores do que os do ECDH e do FFDH.

In [691]:
# ML-KEM-768 params:
N = 256
Q = 3329
K = 3
ETA1 = 2
ETA2 = 2
DU = 10
DV = 4

# # ML-KEM-512 params:
# N_512 = 256
# Q_512 = 3329
# K_512 = 2
# ETA1_512 = 3
# ETA2_512 = 2
# DU_512 = 10
# DV_512 = 4

# # ML-KEM-768 params:
# N_768 = 256
# Q_768 = 3329
# K_768 = 3
# ETA1_768 = 2
# ETA2_768 = 2
# DU_768 = 10
# DV_768 = 4

# # ML-KEM-1024 params:
# N_1024 = 256
# Q_1024 = 3329
# K_1024 = 4
# ETA1_1024 = 2
# ETA2_1024 = 2
# DU_1024 = 11
# DV_1024 = 5



In [692]:
# Definindo o anel Zq
q = 3329  # Este é um primo para o qual 17 é primitivo
Zq = IntegerModRing(q)

def bitrev7(n: int) -> int:
    return int(f"{n:07b}"[::-1], 2)

ZETA = [pow(17, bitrev7(k), Q) for k in range(128)] # used in ntt and ntt_inv
GAMMA = [pow(17, 2*bitrev7(k)+1, Q) for k in range(128)] # used in ntt_mul

    for i in range(256):
        for j in range(256):
            c[i+j] = (c[i+j] + a[j] * b[i]) % q

    # Redução mod X^256 + 1
    for i in range(255):
        c[i] = (c[i] - c[i+256]) % q
    
    # Truncar c
    return c[:256]

# O(n^2) multiplication algorithm for testing/comparison purposes
def poly256_slow_mul(a: List[int], b: List[int]) -> List[int]:
	c = [0] * 511

	for i in range(256):
		for j in range(256):
			c[i+j] = (c[i+j] + a[j] * b[i]) % Q

	# reduction mod X^256 + 1
	for i in range(255):
		c[i] = (c[i] - c[i+256]) % Q
	
	# truncate c
	return c[:256]

    return f_out

# O(n logn)
def ntt(f_in: List[int]) -> List[int]:
	f_out = f_in.copy()
	k = 1
	for log2len in range(7, 0, -1):
		length = 2**log2len
		for start in range(0, 256, 2 * length):
			zeta = ZETA[k]
			k += 1
			for j in range(start, start + length):
				t = (zeta * f_out[j + length]) % Q
				f_out[j + length] = (f_out[j] - t) % Q
				f_out[j] = (f_out[j] + t) % Q
	return f_out


# O(n logn)
def ntt_inv(f_in: List[int]) -> List[int]:
	f_out = f_in.copy()
	k = 127
	for log2len in range(1, 8):
		length = 2**log2len
		for start in range(0, 256, 2 * length):
			zeta = ZETA[k]
			k -= 1
			for j in range(start, start + length):
				t = f_out[j]
				f_out[j] = (t + f_out[j + length]) % Q
				f_out[j + length] = (zeta * (f_out[j + length] - t)) % Q

	for i in range(256):
		f_out[i] = (f_out[i] * 3303) % Q  # 3303 == pow(128, -1, Q)

	return f_out

ntt_add = poly256_add  # it's just elementwise addition

#  O(n)
def ntt_mul(a: List[int], b: List[int]) -> List[int]:
    c = []
    # guarantees that the list has a sufficient size
    a += [0] * (256 - len(a))
    for i in range(128):
        a0, a1 = a[2 * i: 2 * i + 2]
        b0, b1 = b[2 * i: 2 * i + 2]
        c.append((a0 * b0 + a1 * b1 * GAMMA[i]) % q)
        c.append((a0 * b1 + a1 * b0) % q)
    return c

# crypto functions

def mlkem_prf(eta: int, data: bytes, b: int) -> bytes:
	return hashlib.shake_256(data + bytes([b])).digest(64 * eta)

def mlkem_xof(data: bytes, i: int, j: int) -> ShakeStream:
	return ShakeStream(hashlib.shake_128(data + bytes([i, j])).digest)

def mlkem_hash_H(data: bytes) -> bytes:
    # Você pode usar a função de hash SHA3_256 do SageMath
    h = hashlib.sha3_256(data).digest()
    return bytes_to_list(h)

def mlkem_hash_J(data: bytes) -> bytes:
    # Você pode usar a função de hash SHAKE256 do SageMath
    h = hashlib.shake_256(data).digest(32)
    return bytes_to_list(h)

def mlkem_hash_G(data: bytes) -> bytes:
    # Você pode usar a função de hash SHA3_512 do SageMath
    h = hashlib.sha3_512(data).digest()
    return bytes_to_list(h)


In [693]:
# encode/decode logic

def bits_to_bytes(bits: List[int]) -> bytes:
	assert(len(bits) % 8 == 0)
	return bytes(
		sum(bits[i + j] << j for j in range(8))
		for i in range(0, len(bits), 8)
	)

def bytes_to_bits(data: bytes) -> List[int]:
	bits = []
	for word in data:
		for i in range(8):
			bits.append((word >> i) & 1)
	return bits

def byte_encode(d: int, f: List[int]) -> bytes:
	assert(len(f) == 256)
	bits = []
	for a in f:
		for i in range(d):
			bits.append((a >> i) & 1)
	return bits_to_bytes(bits)

def byte_decode(d: int, data: bytes) -> List[int]:
    bits = bytes_to_bits(data)
    num_elements = len(bits) // d
    return [sum(bits[i * d + j] << j for j in range(d)) for i in range(num_elements)]

def compress(d: int, x: List[int]) -> List[int]:
	return [(((n * 2**d) + Q // 2 ) // Q) % (2**d) for n in x]

def decompress(d: int, x: List[int]) -> List[int]:
	return [(((n * Q) + 2**(d-1) ) // 2**d) % Q for n in x]



In [694]:
# sampling

def sample_ntt(xof: ShakeStream):
	res = []
	while len(res) < 256:
		a, b, c = xof.read(3)
		d1 = ((b & 0xf) << 8) | a
		d2 = c << 4 | b >> 4
		if d1 < Q:
			res.append(d1)
		if d2 < Q and len(res) < 256:
			res.append(d2)
	return res


def sample_poly_cbd(eta: int, data: bytes) -> List[int]:
	assert(len(data) == 64 * eta)
	bits = bytes_to_bits(data)
	f = []
	for i in range(256):
		x = sum(bits[2*i*eta+j] for j in range(eta))
		y = sum(bits[2*i*eta+eta+j] for j in range(eta))
		f.append((x - y) % Q)
	return f


# K-PKE

def kpke_keygen(seed: bytes=None) -> Tuple[bytes, bytes]:
	d = os.urandom(32) if seed is None else seed
	ghash = mlkem_hash_G(d)
	rho, sigma = ghash[:32], ghash[32:]

	ahat = []
	for i in range(K):
		row = []
		for j in range(K):
			row.append(sample_ntt(mlkem_xof(rho, i, j)))
		ahat.append(row)
	
	shat = [
		ntt(sample_poly_cbd(ETA1, mlkem_prf(ETA1, sigma, i)))
		for i in range(K)
	]
	ehat = [
		ntt(sample_poly_cbd(ETA1, mlkem_prf(ETA1, sigma, i+K)))
		for i in range(K)
	]
	that = [ # t = a * s + e
		reduce(ntt_add, [
			ntt_mul(ahat[j][i], shat[j])
			for j in range(K)
		] + [ehat[i]])
		for i in range(K)
	]
	print("Type rho", type(rho))
	print("Type that", type(that))
	ek_pke = b"".join(byte_encode(12, [int(a) for a in s]) for s in that) + rho
	dk_pke = b"".join(byte_encode(12, [int(a) for a in s]) for s in shat)
	return ek_pke, dk_pke


def kpke_encrypt(ek_pke: bytes, m: bytes, r: bytes) -> bytes:
    #print("Encrypting...")
    that = [byte_decode(12, ek_pke[i*128*K:(i+1)*128*K]) for i in range(K)]
    rho = ek_pke[-32:]

    # guarantees that the list has a sufficient size
    that += [[0] * 256] * (K - len(that))
    
    # Identical to as in kpke_keygen
    ahat = []
    for i in range(K):
        row = []
        for j in range(K):
            row.append(sample_ntt(mlkem_xof(rho, i, j)))
        ahat.append(row)
    
    rhat = [
        ntt(sample_poly_cbd(ETA1, mlkem_prf(ETA1, r, i)))
        for i in range(K)
    ]
    e1 = [
        sample_poly_cbd(ETA2, mlkem_prf(ETA2, r, i+K))
        for i in range(K)
    ]
    e2 = sample_poly_cbd(ETA2, mlkem_prf(ETA2, r, 2*K))

    u = [ # u = ntt-1(AT*r)+e1
        poly256_add(ntt_inv(reduce(ntt_add, [
            ntt_mul(ahat[i][j], rhat[j]) # i,j are reversed here
            for j in range(K)
        ])), e1[i])
        for i in range(K)
    ]
    mu = decompress(1, byte_decode(1, m))

    # guarantees that the list has a sufficient size
    u += [[0, 0]] * (K - len(u))
    
    v = poly256_add(ntt_inv(reduce(ntt_add, [
        ntt_mul(that[i], rhat[i])
        for i in range(K)
    ])), poly256_add(e2, mu))

    c1 = b"".join(byte_encode(DU, compress(DU, u[i])) for i in range(K))
    c2 = byte_encode(DV, compress(DV, v))
    #print("Encryption complete.")
    return c1 + c2


def kpke_decrypt(dk_pke: bytes, c: bytes) -> bytes:
    #print("Decrypting...")
    c1 = c[:32*DU*K]
    c2 = c[32*DU*K:]
    print("c1", c1)
    print("c2", c2)
    u = [
        decompress(DU, byte_decode(DU, c1[i*32*DU:(i+1)*32*DU]))
        for i in range(K)
    ]
    v = decompress(DV, byte_decode(DV, c2))
    shat = [byte_decode(12, dk_pke[i*384:(i+1)*384]) for i in range(K)]
    w = poly256_sub(v, ntt_inv(reduce(ntt_add, [
        ntt_mul(shat[i], ntt(u[i]))
        for i in range(K)
    ])))
    m = byte_encode(1, compress(1, w))
    #print("Decryption complete.")
    return m

# KEM

def mlkem_keygen(seed1=None, seed2=None):
	z = os.urandom(32) if seed1 is None else seed1
	ek_pke, dk_pke = kpke_keygen(seed2)
	ek = ek_pke
	dk = dk_pke + ek + mlkem_hash_H(ek) + z
	return ek, dk


def mlkem_encaps(ek: bytes, seed=None) -> Tuple[bytes, bytes]:
    m = os.urandom(32) if seed is None else seed
    #print("Mensagem m:", m.hex())
    ghash = mlkem_hash_G(m + mlkem_hash_H(ek))
    k = ghash[:32]
    r = ghash[32:]
    #print("Chave k:", k.hex())
    #print("Vetor de inicialização r:", r.hex())
    c = kpke_encrypt(ek, m, r)
    #print("Texto cifrado c:", c.hex())
    return k, c


def mlkem_decaps(c: bytes, dk: bytes) -> bytes:
    dk_pke = dk[:384*K]
    ek_pke = dk[384*K : 768*K + 32]
    h = dk[768*K + 32 : 768*K + 64]
    z = dk[768*K + 64 : 768*K + 96]
    mdash = kpke_decrypt(dk_pke, c)
    #print("Mensagem descriptografada mdash:", mdash.hex())
    ghash = mlkem_hash_G(mdash + h)
    kdash = ghash[:32]
    rdash = ghash[32:]
    kbar = mlkem_hash_J(z + c)
    cdash = kpke_encrypt(ek_pke, mdash, rdash)
    #print("Texto cifrado cdash:", cdash.hex())
    if cdash != c:
        print("Os textos cifrados não coincidem")
        return kbar
    return kdash


In [695]:
if __name__ == "__main__":
	a = list(range(256))
	b = list(range(1024, 1024+256))

	ntt_res = ntt_inv(ntt_add(ntt(a), ntt(b)))
	poly_res = poly256_add(a, b)
	print("ntt_res:", ntt_res)
	print("poly_res:", poly_res)

	assert(ntt_res == poly_res)

	ntt_prod = ntt_inv(ntt_mul(ntt(a), ntt(b)))
	poly_prod = poly256_slow_mul(a, b)
	print("ntt_prod:", ntt_prod)
	print("poly_prod:", poly_prod)

	assert(ntt_prod == poly_prod)

	ek_pke, dk_pke = kpke_keygen(b"SEED"*8)
	print("ek", ek_pke)
	print("dk", dk_pke)

	msg = b"This is a demonstration message."
	ct = kpke_encrypt(ek_pke, msg, b"RAND"*8)
	print("ct:", ct)
	pt = kpke_decrypt(dk_pke, ct)
	print("pt:", pt)
	#print("Type of pt:", type(pt))
	#print("Type of msg:", type(msg))
	#msg = msg.decode('utf-8') 
	print("pt (hex):", pt.hex())
	print("msg (hex):", msg.hex())

	assert(pt == msg)

	ek, dk = mlkem_keygen()
	k1, c = mlkem_encaps(ek)
	print("encapsulated:", k1.hex())

	k2 = mlkem_decaps(c, dk)
	print("decapsulated:", k2.hex())

	assert(k1 == k2)
	
# if __name__ == "__main__":
    # Teste para ML-KEM-xxx
    # print("Teste para ML-KEM-512:")
    # ek_pke, dk_pke = mlkem_keygen()
    # k1, c = mlkem_encaps(ek_pke)
    # k2 = mlkem_decaps(c, dk_pke)
    
    # assert k1 == k2


    

ntt_res: [1024, 1026, 1028, 1030, 1032, 1034, 1036, 1038, 1040, 1042, 1044, 1046, 1048, 1050, 1052, 1054, 1056, 1058, 1060, 1062, 1064, 1066, 1068, 1070, 1072, 1074, 1076, 1078, 1080, 1082, 1084, 1086, 1088, 1090, 1092, 1094, 1096, 1098, 1100, 1102, 1104, 1106, 1108, 1110, 1112, 1114, 1116, 1118, 1120, 1122, 1124, 1126, 1128, 1130, 1132, 1134, 1136, 1138, 1140, 1142, 1144, 1146, 1148, 1150, 1152, 1154, 1156, 1158, 1160, 1162, 1164, 1166, 1168, 1170, 1172, 1174, 1176, 1178, 1180, 1182, 1184, 1186, 1188, 1190, 1192, 1194, 1196, 1198, 1200, 1202, 1204, 1206, 1208, 1210, 1212, 1214, 1216, 1218, 1220, 1222, 1224, 1226, 1228, 1230, 1232, 1234, 1236, 1238, 1240, 1242, 1244, 1246, 1248, 1250, 1252, 1254, 1256, 1258, 1260, 1262, 1264, 1266, 1268, 1270, 1272, 1274, 1276, 1278, 1280, 1282, 1284, 1286, 1288, 1290, 1292, 1294, 1296, 1298, 1300, 1302, 1304, 1306, 1308, 1310, 1312, 1314, 1316, 1318, 1320, 1322, 1324, 1326, 1328, 1330, 1332, 1334, 1336, 1338, 1340, 1342, 1344, 1346, 1348, 1350, 1352, 